<a href="https://colab.research.google.com/github/Shaddy-joker/GAN/blob/main/SimpleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is done with reference to the Simple GAN implementation by Aladdin Persson https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/GANs/1.%20SimpleGAN/fc_gan.py

In [ ]:
#loading the required libraries
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#defining the dimensions of the MNIST image set
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [ ]:
# function for generator

def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Using only Dense layers for generator

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

In [ ]:
#Defining the discriminator

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [ ]:
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5


    X_train = np.expand_dims(X_train, axis=3)

    half_batch = int(batch_size / 2)



# looping through 1000 epochs to train the discriminator and generator
    for epoch in range(epochs):


        # Training the Discriminator with random images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]


        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generating fake images
        gen_imgs = generator.predict(noise)

        # Training the discriminator on real and fake images
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

    # storing the calculated loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        noise = np.random.normal(0, 1, (batch_size, 100))

        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size


        g_loss = combined.train_on_batch(noise, valid_y)



# print progress

        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))


        if epoch % save_interval == 0:
            save_imgs(epoch)



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# path to store the results of generated images
import os
from os import listdir
path = os.path.join('gdrive', 'MyDrive','dissertation/simpleGAN')

In [ ]:
# function for saving the output images
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(os.path.join(path,"images/mnist_%d.png" % epoch))
    plt.close()



e
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.


discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])


generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise.

z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This only trains the generator and not the discriminator
discriminator.trainable = False

#only take images from generator
valid = discriminator(img)  #Validity check on the generated image


#combine the losses using binary crossentropy
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# training the model for 10000 epochs
train(epochs=10000, batch_size=32, save_interval=500)



generator.save('generator_model.h5')  #Test the model on GAN4_predict...


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 17ms/step
7502 [D loss: 0.701164, acc.: 68.75%] [G loss: 0.923522]
1/1 [==============================] - 0s 24ms/step
7503 [D loss: 0.633943, acc.: 62.50%] [G loss: 0.913878]
1/1 [==============================] - 0s 19ms/step
7504 [D loss: 0.670992, acc.: 65.62%] [G loss: 0.926839]
1/1 [==============================] - 0s 21ms/step
7505 [D loss: 0.652695, acc.: 56.25%] [G loss: 0.812310]
1/1 [==============================] - 0s 20ms/step
7506 [D loss: 0.664972, acc.: 62.50%] [G loss: 0.933143]
1/1 [==============================] - 0s 19ms/step
7507 [D loss: 0.765500, acc.: 34.38%] [G loss: 0.876039]
1/1 [==============================] - 0s 20ms/step
7508 [D loss: 0.683438, acc.: 56.25%] [G loss: 0.970645]
1/1 [==============================] - 0s 24ms/step
7509 [D loss: 0.646519, acc.: 62.50%] [G loss: 0.874670]
1/1 [==============================] - 0s 20ms/step
7510 [D loss: 0.650202, a